## Bitcoin NLP Using Twitter API

Accessing Twitter API

In [18]:
import pickle
import os

In [22]:
if not os.path.exists('secret_twitter_credentials.pkl'):
    Twitter={}
    #Twitter['Header']='POST /1.1/statuses/update.json?include_entities=true HTTP/1.1 \n Accept: */* \n Connection: close \n User-Agent: OAuth python v0.4.4 \n Content-Type: application/x-www-form-urlencoded'
    Twitter['Consumer Key'] = ''
    Twitter['Consumer Secret'] = ''
    Twitter['Access Token'] = ''
    Twitter['Access Token Secret'] = ''
    with open('secret_twitter_credentials.pkl','wb') as f:
        pickle.dump(Twitter, f)
else:
    Twitter=pickle.load(open('secret_twitter_credentials.pkl','rb'))

In [48]:
import twitter

auth = twitter.oauth.OAuth(Twitter['Access Token'],
                           Twitter['Access Token Secret'],
                           Twitter['Consumer Key'],
                           Twitter['Consumer Secret'])

twitter_api = twitter.Twitter(auth=auth)

# Nothing to see by displaying twitter_api except that it's now a
# defined variable

print(twitter_api)

## Use textblob to build a simple sentiment analysis function

In [197]:
from textblob import TextBlob
import re

In [214]:
def get_sentiment (t):
    s = TextBlob(t)
    if s.sentiment.polarity > 0:
        return 'positive'
    elif s.sentiment.polarity == 0:
        return 'neutral'
    else:
        return 'negative'

Access Twitter REST API for tweets

In [201]:
query = input("What subject do you want to analyze for this example? \n")
number = input("How many Tweets do you want to analyze? \n")

What subject do you want to analyze for this example? 
bitcoin -rt
How many Tweets do you want to analyze? 
100


In [232]:
result=twitter_api.search.tweets(q=query, result_type='recent', lang='en', count=number)
statuses = result['statuses']

In [260]:
#print(result)

In [237]:
len(statuses)

91

In [239]:
#print(statuses)

Remove duplicates

In [240]:
all_text = []
filtered_statuses = []
for s in statuses:
    if not s["text"] in all_text:
        filtered_statuses.append(s)
        all_text.append(s["text"])
statuses = filtered_statuses  

In [241]:
len(filtered_statuses)

89

In [242]:
[s['text'] for s in result['statuses']]

['The Low-Key Indicator That The Bitcoin Bubble May Have Burst https://t.co/JcsVepGJII',
 '@alvinfoo @dinisguarda @VanRijmenam @Datafloq @BourseetTrading @chboursin @MHiesboeck @kuriharan @MikeQuindazzi… https://t.co/ypESnOK1c6',
 '@XVGPlankton More interesting than reporting ATM thefts',
 "@Lazy5B My great grandfather doesn't know what an iPhone is, would think artificial intelligence is techno voodoo,… https://t.co/3CdBnutYLA",
 'Earn bitcoin on a daily basis!\n\n1. Follow @slidecoin \n2. Complete instructions in pinned tweet',
 'Bitcoin Bithumb $BTCKRW https://t.co/S0AvysEeB2',
 '@game_musse Right now A LOT of bitcoin.',
 '@_jfeldman @BitcoinKinetics @ClarkCountyNV @inc_bitcoin @BitcoinNevada At the time of their deaths, I was building… https://t.co/KQim6JM6ia',
 '#CYBEREITS #Airdrop still active!!\n\nURL: https://t.co/BTQ3AWntVk …\n\n1. Enter the ETH address\n\n2. Copy the code and… https://t.co/ikUSiSmmLj',
 '[01:37 GMT]Arbitrage opportunity has occurred!!\nThe diff is 0.00000256 

In [268]:
# get the favourite count for the first item
print(json.dumps(statuses[0]['user']['favourites_count'], indent=1))
print(json.dumps(statuses[0], indent=1))

42950
{
 "geo": null,
 "metadata": {
  "iso_language_code": "en",
  "result_type": "recent"
 },
 "place": null,
 "user": {
  "notifications": false,
  "has_extended_profile": true,
  "contributors_enabled": false,
  "id_str": "41140413",
  "profile_use_background_image": true,
  "profile_background_color": "DB46A0",
  "location": "Altcoinworld",
  "followers_count": 92738,
  "profile_banner_url": "https://pbs.twimg.com/profile_banners/41140413/1521716421",
  "utc_offset": 7200,
  "created_at": "Tue May 19 15:43:06 +0000 2009",
  "favourites_count": 42950,
  "url": "https://t.co/eLPIsYqORs",
  "entities": {
   "url": {
    "urls": [
     {
      "expanded_url": "http://bountyhive.io/r/tokens",
      "display_url": "bountyhive.io/r/tokens",
      "url": "https://t.co/eLPIsYqORs",
      "indices": [
       0,
       23
      ]
     }
    ]
   },
   "description": {
    "urls": []
   }
  },
  "profile_link_color": "DB46A0",
  "profile_background_image_url_https": "https://pbs.twimg.com/pro

In [294]:
status_texts = [ status['text'] 
                 for status in statuses ]

screen_names = [ user_mention['screen_name'] 
                 for status in statuses
                     for user_mention in status['entities']['user_mentions'] ]

hashtags = [ hashtag['text'] 
             for status in statuses
                 for hashtag in status['entities']['hashtags'] ]

fav = [ status['user']['favourites_count']
        for status in statuses]


# Compute a collection of all words from all tweets
words = [ w 
          for t in status_texts 
              for w in t.split() ]

In [295]:
# Explore the first 5 items for each...

print(json.dumps(status_texts[0], indent=1))
print(json.dumps(screen_names[0], indent=1)) 
print(json.dumps(hashtags[0], indent=1))
print(json.dumps(words[0], indent=1))
print(json.dumps(fav[0], indent=1))

"The Low-Key Indicator That The Bitcoin Bubble May Have Burst https://t.co/JcsVepGJII"
"alvinfoo"
"CYBEREITS"
"The"
42950


In [296]:
print(json.dumps(status_texts[0], indent=1))

"The Low-Key Indicator That The Bitcoin Bubble May Have Burst https://t.co/JcsVepGJII"


In [298]:
print(get_sentiment(status_texts[0]))

neutral


In [300]:
from collections import Counter

for item in [words,hashtags]:
    c = Counter(item)
    print(c.most_common()[:10]) # top 10
    print()

[('the', 28), ('Bitcoin', 25), ('of', 23), ('a', 22), ('on', 18), ('to', 17), ('and', 17), ('in', 16), ('#Bitcoin', 12), ('I', 11)]

[('Bitcoin', 12), ('Cybersecurity', 10), ('bitcoin', 8), ('ethereum', 5), ('blockchain', 3), ('crypto', 3), ('Blockchain', 2), ('news', 2), ('eth', 2), ('Opirium', 1)]



In [302]:
retweets = [
            # Store out a tuple of these three values ...
            (status['retweet_count'], 
             status['retweeted_status']['user']['screen_name'],
             status['text'].replace("\n","\\")) 
            
            # ... for each status ...
            for status in statuses 
            
            # ... so long as the status meets this condition.
                if 'retweeted_status' in status
           ]

In [306]:
print(retweets)

[]
